# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [2]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [3]:
from computions import *

### get_describe_nominal function test

Basic situation

In [4]:
get_describe_nominal(test_frame.iloc[:,1], Y)

,object_variable,1.0,1.0%,0.0,0.0%,2.0,2.0%
2.0,107,30.0,28.037383,77.0,71.962617,0.0,0.000000
9.0,106,59.0,55.660377,0.0,0.000000,47.0,44.339623
3.0,105,53.0,50.476190,52.0,49.523810,0.0,0.000000
6.0,104,93.0,89.423077,8.0,7.692308,3.0,2.884615
7.0,102,86.0,84.313725,0.0,0.000000,16.0,15.686275
8.0,99,64.0,64.646465,0.0,0.000000,35.0,35.353535
5.0,98,87.0,88.775510,8.0,8.163265,3.0,3.061224
1.0,86,6.0,6.976744,80.0,93.023256,0.0,0.000000
4.0,82,54.0,65.853659,28.0,34.146341,0.0,0.000000
10.0,56,15.0,26.785714,0.0,0.000000,41.0,73.214286


Data frame with emptys

In [5]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['object_variable'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['object_variable'].sum() == n))

Is all obs. are in table -  False
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [6]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.5070038863396976, 0.0: 0.3267819608137527, 2.0: 0.7837951199838677}
{'category 1': 0.5070038863396976, 'cetegory 0': 0.3267819608137527, 'category 2': 0.7837951199838677}
{'category 1': 0.5103769820434737, 'cetegory 0': 0.32596445811273744, 'category 2': 0.7814436301236697}
{1.0: 0.6732180391862473, 0.0: 0.3267819608137527}


AUC nominal

In [7]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys')
print(get_AUC_numeric(na_containts_frame['object_variable'], Y_strs))

{'category 1': 0.8264182314934763, 'cetegory 0': 0.9431958937016741, 'category 2': 0.9118531962089131}
AUC bynary
{1.0: 0.943195893701674, 0.0: 0.9431958937016741}
AUC with emptys
{'category 1': 0.6748342728392415, 'cetegory 0': 0.05636097642125659, 'category 2': 0.9124424350398334}


Get publish AUC for any predictor type

In [8]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)


with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

nominal predictor without table
{1.0: {'AUC': 0.8264182314934763, 'rel_type': 1, 'GINI': 0.6528364629869525}, 0.0: {'AUC': 0.9431958937016741, 'rel_type': 1, 'GINI': 0.8863917874033482}, 2.0: {'AUC': 0.9118531962089131, 'rel_type': 1, 'GINI': 0.8237063924178263}}
moninal predictor with table
{'category 1': {'AUC': 0.8264182314934763, 'rel_type': 1, 'GINI': 0.6528364629869525}, 'cetegory 0': {'AUC': 0.9431958937016741, 'rel_type': 1, 'GINI': 0.8863917874033482}, 'category 2': {'AUC': 0.9118531962089131, 'rel_type': 1, 'GINI': 0.8237063924178263}}
numeric predictor
{1.0: {'AUC': 0.5070038863396976, 'rel_type': 1, 'GINI': 0.014007772679395236}, 0.0: {'AUC': 0.6732180391862472, 'rel_type': -1, 'GINI': 0.34643607837249446}, 2.0: {'AUC': 0.7837951199838677, 'rel_type': 1, 'GINI': 0.5675902399677355}}


## Writing to excel

Adding a new sheet

In [9]:
xl_writer = pd.ExcelWriter("test_result/sheet_creation.xlsx",engine='xlsxwriter')


for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

Adding a info about different predictors

In [10]:
xl_writer = pd.ExcelWriter("test_result/write_to_book.xlsx",engine='xlsxwriter')

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.write_to_book(xl_writer)
xl_writer.close()

UnboundLocalError: local variable 'header_info' referenced before assignment